<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/202210_Bigdata/08_Spak_SQL%2C_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic")
sc   = SparkContext(conf=conf)

# RDD만을 이용한 데이터 추출

In [ ]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

`마블` 영화 중 관객 수가 500만 이상인 영화를 가져오기
1. `Inner Join` -> `Filter By Movie` -> `Filter By attendance`
2. `Filter By Movie`, `Filter By attendance` -> `Inner Join`

In [ ]:
# CASE1. join 먼저, filter 나중에
movie_att = movies_rdd.join(attendances_rdd)
movie_att.filter(
    lambda x : x[1][0][1] == "마블" and x[1][1][0] > 5000000
).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [ ]:
# CASE 2. filter 먼저, join 나중에
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == '마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

- 동일한 결과지만 filter를 먼저 수행해서 가져올 데이터를 걸러 낸 다음 join을 하는 CASE 2가 훨씬 효율적이다.

**하지만** 매번 이런 고민을 한다면...? 

# Spark SQL 사용해 보기

## 1. SparkSession 만들기
- SparkContext에 해당하며, 새로운 스파크 어플리케이션을 생성

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

RDD를 준비

In [ ]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [ ]:
movie_schema = ["id", "name", "company", "year", "month", "day"]

## 2. 데이터 프레임 만들기

In [ ]:
# 스파크가 알아서 데이터 타입을 결정을 지어 줍니다.
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [ ]:
# 스키마의 타입 확인
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

전체 데이터 프레임 내용을 확인
- show()


In [ ]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



- `SELECT` : 컬럼 조회 하기위한 쿼리 절
- `  FROM` : 테이블. (어떤 데이터프레임(테이블)에서 데이터를 가지고 올 것인가)
- ` WHERE` : 데이터가 조회되기 위한 조건

★데이터프레임을 temporary view에 등록해야 spark sql을 사용할 수 있게 된다.★

In [ ]:
df.createOrReplaceTempView("movies")

영화 이름만 가져오기

In [ ]:
query = """

SELECT name
  FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



2010년 이후에 개봉한 영화를 조회

In [ ]:
query = """

SELECT *
  FROM movies
 WHERE year >= 2010
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [ ]:
# 2012년도 이전에 개봉한 영화의 이름과 회사를 출력
query = """

SELECT name,
       company
       
  FROM movies
 WHERE year <= 2012
 
"""
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



In [ ]:
# like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때
# % 기호를 사용해서 문장이 매칭되는지 확인 가능!


# 제목이 ~~맨으로 끝나는 데이터의 모든 정보를 조회
query = """

SELECT *
  FROM movies
 WHERE name LIKE '%맨'
 
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# '이'가 들어간 영화 찾기
query = """

SELECT *
  FROM movies
 WHERE name LIKE '%이%'
 
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# BETWEEN 특정 데이터와 데이터 사이를 조회

# 개봉 월이 4 ~ 8월 사이. 4 <= 개봉월 <= 8

query = """

SELECT *
  FROM movies
 WHERE month BETWEEN 4 AND 8
 
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# 이름이 ~맨으로 끝나고, 개봉연도가 2010년 이하인 영화
query = """

SELECT *
  FROM movies
 WHERE name LIKE '%맨'
   AND year <= 2010
 
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# 영화의 회사가 마블 또는 DC인 영화
query = """

SELECT *
  FROM movies
 WHERE company = '마블' OR company='DC'
 
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# in 연산을 활용해 보기
# 컬럼명 in (값1, 값2, ...)

query = """

SELECT *
  FROM movies
 WHERE company in ('마블', 'DC')
 
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# 회사가 "마"로 시작을 하거나, "니"로 끝나는 영화
query = """

SELECT *
  FROM movies
 WHERE company LIKE '마%' OR company LIKE '%니'
 
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# 회사가 "마"로 시작을 하거나, "니"로 끝나는 영화 중 2010년 이후로 개봉한 영화
query = """

SELECT *
  FROM movies
 WHERE ( company LIKE '마%' OR company LIKE '%니' )
   AND year >= 2010
 
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



`ORDER BY`절 : 정렬
- `asc` : ascending. 오름차순 (기본값. 생략이 가능)
- `desc` : descending : 내림차순

In [ ]:
# 개봉 연도 오름차순으로 확인
query = """
SELECT *
  FROM movies
 ORDER BY year ASC
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [ ]:
# 개봉 연도 오름차순으로 확인
query = """
SELECT *
  FROM movies
 ORDER BY year DESC
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  1|어벤져스|   마블|2012|    4| 26|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



- `count` : 개수 세기
- `mean` : 평균 구하기
- `sum` : 총 합

In [ ]:
query = """
SELECT count(*) as movie_count
  FROM movies
  WHERE company = "DC"
"""

spark.sql(query).show()

+-----------+
|movie_count|
+-----------+
|          2|
+-----------+



# Join 구현하기

In [ ]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

직접 스키마 지정해 보기

In [ ]:
from pyspark.sql.types import StringType, FloatType, IntegerType, StructType, StructField

In [ ]:
att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [ ]:
att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [ ]:
att_df.createOrReplaceTempView("att")

In [ ]:
# 쿼리를 사용하지 않고 모든 데이터를 확인
#  DataFrame API를 사용
att_df.select("*").show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [ ]:
query = """
SELECT movies.id, movies.name, movies.company, att.att
  FROM movies
  JOIN att ON movies.id = att.id
"""

spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3934592E7|
|  2|  슈퍼맨|     DC|  2182227.0|
|  3|  배트맨|     DC|  4226242.0|
|  4|겨울왕국| 디즈니|1.0303058E7|
|  5|아이언맨|   마블|  4300365.0|
+---+--------+-------+-----------+



# 데이터 프레임 API

In [ ]:
# select
# data frame api의 결과는 Transformations 작업이기 때문에 collect, show를 통해서 데이터를 확인
df.select("*").collect()

[Row(id=1, name='어벤져스', company='마블', year=2012, month=4, day=26),
 Row(id=2, name='슈퍼맨', company='DC', year=2013, month=6, day=13),
 Row(id=3, name='배트맨', company='DC', year=2008, month=8, day=6),
 Row(id=4, name='겨울왕국', company='디즈니', year=2014, month=1, day=16),
 Row(id=5, name='아이언맨', company='마블', year=2008, month=4, day=30)]

In [ ]:
df.select("name", "company").collect()

[Row(name='어벤져스', company='마블'),
 Row(name='슈퍼맨', company='DC'),
 Row(name='배트맨', company='DC'),
 Row(name='겨울왕국', company='디즈니'),
 Row(name='아이언맨', company='마블')]

In [ ]:
df.select(df.name, (df.year-2000).alias("year")).show()

+--------+----+
|    name|year|
+--------+----+
|어벤져스|  12|
|  슈퍼맨|  13|
|  배트맨|   8|
|겨울왕국|  14|
|아이언맨|   8|
+--------+----+



In [ ]:
# agg : Aggreagte의 약자로써, 그룹핑 후 데이터를 하나로 합쳐주는 역할
df.agg({"id": "count"}).collect()

[Row(count(id)=5)]

In [ ]:
query = """
SELECT count(*) as movie_count
  FROM movies
  WHERE company = "DC"
"""

spark.sql(query).collect()

[Row(movie_count=2)]

In [ ]:
from pyspark.sql import functions as F
df.agg(F.min(df.year)).collect()

[Row(min(year)=2008)]

In [ ]:
df.groupBy().avg().collect()

[Row(avg(id)=3.0, avg(year)=2011.0, avg(month)=4.6, avg(day)=18.2)]

In [ ]:
# 회사별 개봉월의 평균
df.groupBy('company').agg({"month": "mean"}).collect()

[Row(company='디즈니', avg(month)=1.0),
 Row(company='마블', avg(month)=4.0),
 Row(company='DC', avg(month)=7.0)]

In [ ]:
# 회사 별 월 별 영화 개수 정보
df.groupBy([df.company, df.month]).count().collect()

[Row(company='디즈니', month=1, count=1),
 Row(company='DC', month=8, count=1),
 Row(company='DC', month=6, count=1),
 Row(company='마블', month=4, count=2)]

In [ ]:
# join : 다른 데이터 프레임과 사용자가 지정한 컬럼을 기준으로 합치는 작업
df.join(att_df, 'id').select(df.name, att_df.att).show()

+--------+-----------+
|    name|        att|
+--------+-----------+
|어벤져스|1.3934592E7|
|  슈퍼맨|  2182227.0|
|  배트맨|  4226242.0|
|겨울왕국|1.0303058E7|
|아이언맨|  4300365.0|
+--------+-----------+



In [ ]:
# select, where, orderBy 절 사용
marvel_df = df.select("name", "company", "year").where("company=='마블'").orderBy("id")
marvel_df.collect()

[Row(name='어벤져스', company='마블', year=2012),
 Row(name='아이언맨', company='마블', year=2008)]

In [ ]:
spark.stop()
sc.stop()